## SentencePiece Tokenizer for Legal Texts Corpus

In [ ]:
"""
PIPELINE

	PRE PROCESSING
1. Pour chaque fichier CSV:
- Tout en minuscule
- Suppression caractères spéciales
- Suppression des espaces 
- Suppression des élisions
- Stemmisation
- Suppression des "stop words"

	TRAINING AND TEST CORPUS
1. Garder 4/5 fichier CSV pour le modèle de tokenisation
2. Tester sur 1 fichier CSV
--> Création de fichiers textes représentant le corpus

	ANALYSE DU CORPUS POUR DETERMINER LA TAILLE DU VOCABULAIRE
1. Comptage des mots
2. Décider de la taille du vocabulaire en fonction d'une couverture désirée
--> Ici pour couvrir 97%, on choisit une taille du vocabulaire un peu en dessous de 20K mots.

	ENTRAINEMENT DU TOKENIZER SENTENCE PIECE
	ENCODAGE DU FICHIER TEST

	ANALYSE QUANTITATIVE
1. Utilisation du vocabulaire (bien environ égal à 97%)
2. Longueur moyenne d'un token (si la taille est trop petite cela pourrait indiquer trop de segmentation : 5.67 parait raisonnable
3. OOV rate
4. Caractéristiques diverses des corpus
5. Calcul de l'overlap de vocabulaire entre training et test sets: 64%, ne semble pas suffisant, on devrait augmenter le coverage à 99% voir plus pour bien inclure tous les mots même les rares.

	ANALYSE QUALITATIVE
Commentaires intégrés au notebook.



"""

In [1]:
# Importation des librairies utiles

import numpy as np 
import pandas as pd 
import re
import sentencepiece as spm
import io
from collections import Counter
import random
from collections import Counter

# On load la librairie spacy pour avoir accès à une liste de stop words 
import spacy

# On utilise cette librairie pour faire une stemmisation avant d'utiliser SentencePiece
from nltk.stem.snowball import FrenchStemmer



In [2]:
file_paths = ['datas/jorf_2019.csv', 'datas/jorf_2020.csv', 'datas/jorf_2021.csv', 'datas/jorf_2022.csv', 'datas/jorf_2023.csv']

In [3]:
nlp = spacy.load("fr_core_news_sm")
french_stopwords = nlp.Defaults.stop_words

In [4]:
stemmer = FrenchStemmer()

In [5]:
def preprocess_text(text):
    text = text.lower() # tout en minuscule
    text = re.sub(r'\W+', ' ', text)  # enlèvement des caractères spéciales
    text = re.sub(r'\s+', ' ', text).strip()  # enlèvement des espaces
    # Suppression des élisions
    text = re.sub(r"l'|t'|qu'|n'|s'|c'|j'|d'|m'|jusqu'|lorsqu'|puisqu'", '', text)

    # Suppression des mots vides
    words = text.split()
    filtered_stemmed_words = [stemmer.stem(word) for word in words if word not in french_stopwords]
    
    return ' '.join(filtered_stemmed_words)

train_texts = []
for file in file_paths[:-1]:
    df = pd.read_csv(file, sep='|')
    df['text'] = df[df.columns[-1]].apply(preprocess_text)
    train_texts.extend(df['text'].tolist())

test_df = pd.read_csv(file_paths[-1], sep='|')
test_df['text'] = test_df[test_df.columns[-1]].apply(preprocess_text)
test_texts = []
test_texts.extend(test_df['text'].tolist())

In [39]:
"""
SAMPLING

On supprime cette étape après avoir posé la question du sampling en cours
Les textes juridiques contiennent beaucoup de mots rares qui sont précieux pour la compréhension

# Garder que 25% pour se focus sur les phrases avec la plus grosse distribution
sample_size = int(0.25 * len(train_texts))
train_texts_sampled = random.sample(train_texts, sample_size)

"""

In [6]:
# Après avoir rassemblé les textes on crée le corpus sur lequel SentencePiece créera la tokenisation
# On garde un csv file pour le test

with open('train_data.txt', 'w', encoding='utf-8') as f:
    for text in train_texts:
        f.write(text + '\n')

with open('test_data.txt', 'w', encoding='utf-8') as f:
    for text in test_texts:
        f.write(text + '\n')

In [7]:
# Analyser le corpus pour déterminer la taille du vocabulaire
all_words = ' '.join(train_texts).split()
word_counts = Counter(all_words)
total_words = sum(word_counts.values())
cumulative_coverage = 0
desired_coverage = 0.97  # Par exemple, couvrir 97% du corpus
vocab_size = 0

for word, count in word_counts.most_common():
    cumulative_coverage += count / total_words
    vocab_size += 1
    if cumulative_coverage >= desired_coverage:
        break

print(f"Taille de vocabulaire suggérée pour couvrir {desired_coverage*100}% du corpus : {vocab_size}")


Taille de vocabulaire suggérée pour couvrir 97.0% du corpus : 19139


In [8]:
# Entrainement 
spm.SentencePieceTrainer.train(input='train_data.txt', model_prefix='spm_model', vocab_size=vocab_size, model_type='unigram')

# Charger le modèle
sp = spm.SentencePieceProcessor()
sp.load('spm_model.model')

True

In [9]:
with open('test_data.txt', 'r', encoding='utf-8') as f:
    test_texts = f.readlines()

tokenized_test_texts = [sp.encode_as_pieces(text.strip()) for text in test_texts]

In [10]:
# Evaluation
"""
Usage du vocabulaire: vérifier le pourcentage de vocabulaire utilisé dans le test set 
Taille moyenne d'un token: si la taille est trop petite, cela pourrait indiquer trop de segmentation des mots
Out of Vocabulary Rate: on aimerait qu'il soit le plus bas possible
"""

all_tokens = [token for text in tokenized_test_texts for token in text]

vocab_usage = len(set(all_tokens)) / sp.get_piece_size()
print(f"Vocabulary Usage: {vocab_usage:.2%}")

avg_token_length = sum(len(token) for token in all_tokens) / len(all_tokens)
print(f"Average Token Length: {avg_token_length:.2f}")

oov_tokens = all_tokens.count('<unk>') # est-ce le bon marqueur?
oov_rate = oov_tokens / len(all_tokens)
print(f"OOV Rate: {oov_rate:.2%}")

Vocabulary Usage: 96.30%
Average Token Length: 5.67
OOV Rate: 0.00%


In [11]:
def get_word_statistics(texts):
    words = [word for sentence in texts for word in sentence.split()]
    word_counts = Counter(words)
    n_unique_words = len(word_counts)
    n_total_words = sum(word_counts.values())
    avg_word_frequency = np.mean(list(word_counts.values()))
    return n_unique_words, n_total_words, avg_word_frequency, word_counts

def get_sentence_lengths(texts):
    sentence_lengths = [len(sentence.split()) for sentence in texts]
    avg_sentence_length = np.mean(sentence_lengths)
    return sentence_lengths, avg_sentence_length

In [12]:
train_unique_words, train_total_words, train_avg_word_freq, train_word_counts = get_word_statistics(train_texts)
train_sentence_lengths, train_avg_sentence_length = get_sentence_lengths(train_texts)

print("Training Corpus:")
print(f"Unique Words: {train_unique_words}")
print(f"Total Words: {train_total_words}")
print(f"Average Word Frequency: {train_avg_word_freq:.2f}")
print(f"Average Sentence Length: {train_avg_sentence_length:.2f}")

Training Corpus:
Unique Words: 304713
Total Words: 25093174
Average Word Frequency: 82.35
Average Sentence Length: 14.02


In [13]:
test_unique_words, test_total_words, test_avg_word_freq, test_word_counts = get_word_statistics(test_texts)
test_sentence_lengths, test_avg_sentence_length = get_sentence_lengths(test_texts)

print("\nTest Corpus:")
print(f"Unique Words: {test_unique_words}")
print(f"Total Words: {test_total_words}")
print(f"Average Word Frequency: {test_avg_word_freq:.2f}")
print(f"Average Sentence Length: {test_avg_sentence_length:.2f}")


Test Corpus:
Unique Words: 114961
Total Words: 6149586
Average Word Frequency: 53.49
Average Sentence Length: 13.54


In [14]:
shared_vocab = set(train_word_counts.keys()).intersection(set(test_word_counts.keys()))
overlap_percentage = len(shared_vocab) / min(train_unique_words, test_unique_words)
print(f"\nVocabulary Overlap between Training and Test: {overlap_percentage:.2%}")


Vocabulary Overlap between Training and Test: 64.25%


In [15]:
sampled_sentences = random.sample(train_texts + test_texts, 10)

for sentence in sampled_sentences:
    tokenized = sp.encode_as_pieces(sentence)
    print(f"Original: {sentence}")
    print(f"Tokenized: {tokenized}\n")

Original: 1 décret n 76 1073 22 novembr 1976 relat mis protect judiciair travail d intérêt général prononc juridict mineur
Tokenized: ['▁1', '▁décret', '▁n', '▁76', '▁10', '73', '▁22', '▁novembr', '▁1976', '▁relat', '▁mis', '▁protect', '▁judiciair', '▁travail', '▁d', '▁intérêt', '▁général', '▁prononc', '▁juridict', '▁mineur']

Original: command second d group gendarmer transport aérien

Tokenized: ['▁command', '▁second', '▁d', '▁group', '▁gendarmer', '▁transport', '▁aérien']

Original: barem national détermin titr indiqu anné conseil supérieur rémuner contrôleur
Tokenized: ['▁barem', '▁national', '▁détermin', '▁titr', '▁indiqu', '▁anné', '▁conseil', '▁supérieur', '▁rémuner', '▁contrôleur']

Original: lorsqu originair proven pay non membr l union européen envois d animal produit d origin animal produit germinal produit animal produit dériv derni d aliment animal micro organ pathogen animal produit susceptibl véhicul apparten l catégor mention l articl 47 regl ue 2017 625 15 mar 2017 sou

ANALYSE QUALITATIVE

- Subword: la stemmisation semble avoir fonctionnée
- Nombres: les nombres sont gardés entiers dans les tokens ce qui est important dans ce type de corpus puisque ces nombres ont des significations bien précises (date, numéro d'article etc.)
- Les mots comme "caractéristiques", "administration" ou "réalité" sont gardés comme un seul token ce qui préserve leur sémantique. 
- "oeuvre" est correctemment tokenisé donc les caractères spéciales ne représentent pas un problème
- Quelques bizzareries: "îles" -> "î" + "les"

In [16]:
# Analyse des 50 mots les plus fréquents de plus de 4 lettres

# Chargement de votre modèle SentencePiece
sp = spm.SentencePieceProcessor()
sp.load('spm_model.model')  # Assurez-vous que le chemin est correct

# Concaténer tous les textes de votre corpus
all_text = ' '.join(train_texts)  # ou test_texts selon le corpus que vous voulez analyser

# Compter la fréquence des mots
word_freq = Counter(all_text.split())

# Filtrer les mots de plus de 4 lettres et obtenir les 50 plus fréquents
filtered_words = [word for word in word_freq if len(word) > 4]
top_50_words = sorted(filtered_words, key=lambda word: word_freq[word], reverse=True)[:50]

# Tokeniser et afficher les mots avec leurs tokenisations
for word in top_50_words:
    tokenized = sp.encode_as_pieces(word)
    print(f"Mot: {word}, Tokenisé: {tokenized}")

Mot: articl, Tokenisé: ['▁articl']
Mot: arrêt, Tokenisé: ['▁arrêt']
Mot: présent, Tokenisé: ['▁présent']
Mot: servic, Tokenisé: ['▁servic']
Mot: ministr, Tokenisé: ['▁ministr']
Mot: général, Tokenisé: ['▁général']
Mot: charg, Tokenisé: ['▁charg']
Mot: disposit, Tokenisé: ['▁disposit']
Mot: administr, Tokenisé: ['▁administr']
Mot: décret, Tokenisé: ['▁décret']
Mot: mention, Tokenisé: ['▁mention']
Mot: compt, Tokenisé: ['▁compt']
Mot: national, Tokenisé: ['▁national']
Mot: aliné, Tokenisé: ['▁aliné']
Mot: professionnel, Tokenisé: ['▁professionnel']
Mot: remplac, Tokenisé: ['▁remplac']
Mot: arret, Tokenisé: ['▁arret']
Mot: relat, Tokenisé: ['▁relat']
Mot: publiqu, Tokenisé: ['▁publiqu']
Mot: établ, Tokenisé: ['▁établ']
Mot: appliqu, Tokenisé: ['▁appliqu']
Mot: officiel, Tokenisé: ['▁officiel']
Mot: prévu, Tokenisé: ['▁prévu']
Mot: journal, Tokenisé: ['▁journal']
Mot: condit, Tokenisé: ['▁condit']
Mot: candidat, Tokenisé: ['▁candidat']
Mot: modifi, Tokenisé: ['▁modifi']
Mot: français, Toke

Les mots les plus fréquemment utilisés sont tokenisés en unité distincte ce qui suggère que le modèle a appris efficacement à reconnaitre ces mots communs.